<h1>Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2). </h1>

<h2>Report (15 marks) </h2>

<h2>Introduction<h5>
    
London is the capital city of the UK and has a population of approximately 9M. It is a global city and is the world's most visited as measured by airport arrivals. It is an economic powerhouse both for the UK (representing approximately a quarter of the UK GDP) and a cultural destination - including landmarks such as the Tower of London,. Kew Gardens, The Palace of Westmnster and the Royal Observatory at Greenwich amongst many others. More than 300 languages are spoken in the metropolitan area. To summarise the city is large, diverse and wealthy.

As such, it offers multiple business opportunities including in the leisure, hospitality and tourism sectors. The pizza category is expected to grow globally over the next 5 years and therefore I believe that there is an opportunity to open new pizza restaurants in London. Interested parties in this research would include potential investors and the management of existing pizza restaurant operators.
    
The business problem is that I want to expand a pizza restaurant business to 5 new areas in London, UK. I want to recommend to management the best 10 postcode areas to explore based on the following criteria:<h5>
    
1. The number of other pizza restaurants in the same postcode area (to be clear, I want there to be some pizza restaurants to indicate that there is some demand for the category, but not too many which would indicate that the competition is too fierce.)
    
2. The number of other restaurants in the area (to be clear, I want there to be some restaurants to indicate demand, but not too many, indicating fierce competition.)

<h2>Data<h5>
    
Based on the modeling I will identify the target 10 areas for investigation and the  next steps will be to identify specific locations, rental and fitout costs and demand estimates to select the final 5 proposed locations.
    
1. Postcode information with geo-co-ordinates can be sourced from <a href="https://www.doogal.co.uk/AdministrativeAreas.php?district=E09000001">HERE</a> and can be downloaded in a CSV format.
    
2. The number of other restaurants and the number of pizza restaurants will be sourced from the Foursquare API.
    
<h2>Methodology<h5>
    
1. The postcode data will be wrangled into a dataframe - including only live postal codes and their geo-co-ordinates from a .csv file.
    
2. For each live postcode, data will be called from the Foursquare API with the number of pizza restaurants and other restaurants  
    
4. Based on the output the postcodes will be ranked and the top 10 selected for further investigation.
    
<h2>Results<h5>
    
1. The top postcodes are as follows:
    E17JF,
    E17HY,
    EC2M4QB,
    EC2M4PT,
    EC2M7PY,
    E17HP,
    E17JH,
    E17JJ,
    E17LS,
    EC2M3TP


<h2>Discussion<h5>

1. Based on this, initial analysis, I would recommend that the next steps should be a review of the areas in person to understand the likely footfall, and engaging with realtors to understand better the rental market.
    
<h2>Conclusion<h5>
    
<h5>2. There is certainly scope for pizza restaurants in the areas indicated.

In [1]:
# import all necessary libraries


import requests # library to handle requests
import random # library for random number generation
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [2]:
# Read the imported csv as the dataframe 
df = pd.read_csv('City of London postcodes.csv', header = None)

# Define Headers
headers = ["Postcode","In Use","Latitude","Longitude","Easting","Northing","Grid Ref","Ward","Parish","Introduced","Terminated","Altitude","Country","Last Updated","Quality","LSOA Code","LSOA Name"]
df.columns = headers

print (df.shape)

df.head(10)

(6799, 17)


,Postcode,In Use,Latitude,Longitude,Easting,Northing,Grid Ref,Ward,Parish,Introduced,Terminated,Altitude,Country,Last Updated,Quality,LSOA Code,LSOA Name
0,E1 6AN,Yes,51.518895,-0.078378,533425,181747,TQ334817,Bishopsgate,"City of London, unparished area",01/01/1980,NaN,32,England,03/06/2020,Within the building of the matched address clo...,E01032739,City of London 001F
1,E1 7AA,Yes,51.515567,-0.075635,533625,181382,TQ336813,Portsoken,"City of London, unparished area",01/12/2000,NaN,28,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
2,E1 7AD,Yes,51.515457,-0.076718,533550,181368,TQ335813,Portsoken,"City of London, unparished area",01/09/2013,NaN,31,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
3,E1 7AE,Yes,51.515613,-0.076899,533537,181385,TQ335813,Portsoken,"City of London, unparished area",01/07/2013,NaN,30,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
4,E1 7AF,Yes,51.515613,-0.076899,533537,181385,TQ335813,Portsoken,"City of London, unparished area",01/01/2013,NaN,30,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
5,E1 7AW,Yes,51.515630,-0.076279,533580,181388,TQ335813,Portsoken,"City of London, unparished area",01/01/1980,NaN,29,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
6,E1 7AX,Yes,51.515526,-0.078592,533420,181372,TQ334813,Aldgate,"City of London, unparished area",01/05/2017,NaN,16,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
7,E1 7AY,Yes,51.515526,-0.078592,533420,181372,TQ334813,Aldgate,"City of London, unparished area",01/09/2017,NaN,17,England,03/06/2020,Imputed by ONS by reference to surrounding pos...,E01000005,City of London 001E
8,E1 7BH,Yes,51.515175,-0.077610,533489,181335,TQ334813,Aldgate,"City of London, unparished area",01/01/1980,NaN,31,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E
9,E1 7BS,Yes,51.515432,-0.076806,533544,181365,TQ335813,Portsoken,"City of London, unparished area",01/01/1980,NaN,31,England,03/06/2020,Within the building of the matched address clo...,E01000005,City of London 001E


In [3]:
#Drop Columns that are not in use

df2 = df.drop(labels=['Easting','Northing','Grid Ref','Parish','Introduced','Terminated','Altitude','Country','Last Updated','Quality','LSOA Code','LSOA Name'], axis=1, index=None, columns=None, level=None, inplace=False, errors='raise')

In [4]:
print (df2.shape)
df2.head(20)

(6799, 5)


,Postcode,In Use,Latitude,Longitude,Ward
0,E1 6AN,Yes,51.518895,-0.078378,Bishopsgate
1,E1 7AA,Yes,51.515567,-0.075635,Portsoken
2,E1 7AD,Yes,51.515457,-0.076718,Portsoken
3,E1 7AE,Yes,51.515613,-0.076899,Portsoken
4,E1 7AF,Yes,51.515613,-0.076899,Portsoken
5,E1 7AW,Yes,51.515630,-0.076279,Portsoken
6,E1 7AX,Yes,51.515526,-0.078592,Aldgate
7,E1 7AY,Yes,51.515526,-0.078592,Aldgate
8,E1 7BH,Yes,51.515175,-0.077610,Aldgate
9,E1 7BS,Yes,51.515432,-0.076806,Portsoken


In [5]:
# Drop Postcodes that are not in use
df2.drop(df2.index[df['In Use'] == 'No'], inplace = True)


#Remove duplicates (if any)
df2 = df2.drop_duplicates()

# Drop In Use column
df2 = df2.drop(labels=['In Use'], axis=1, index=None, columns=None, level=None, inplace=False, errors='raise')
# Reset the index 
df2 = df2.reset_index()
del df2['index']


print (df2.shape)
df2.head(20)

(1707, 4)


,Postcode,Latitude,Longitude,Ward
0,E1 6AN,51.518895,-0.078378,Bishopsgate
1,E1 7AA,51.515567,-0.075635,Portsoken
2,E1 7AD,51.515457,-0.076718,Portsoken
3,E1 7AE,51.515613,-0.076899,Portsoken
4,E1 7AF,51.515613,-0.076899,Portsoken
5,E1 7AW,51.515630,-0.076279,Portsoken
6,E1 7AX,51.515526,-0.078592,Aldgate
7,E1 7AY,51.515526,-0.078592,Aldgate
8,E1 7BH,51.515175,-0.077610,Aldgate
9,E1 7BS,51.515432,-0.076806,Portsoken


In [83]:
#Define Latitiude and Longitude
latitude = df2.loc[0, 'Latitude']
longitude = df2.loc[0, 'Longitude']

#Define Foursquare ID and Secret
CLIENT_ID = 'E3RGLK4WEGWRU1EJVZC1KHVAFKLG0D3HAY1TCMOMAV31IXAC' # your Foursquare ID
CLIENT_SECRET = 'LK2V3N010RFQMUJJJXRMQ4LADNDLBLKBYFU3LN3I2KWIGZRB' # your Foursquare Secret
VERSION = 20200620
latitude = df2.loc[0, 'Latitude']
longitude = df2.loc[0, 'Longitude']
search_query = 'Restaurant'
radius = 1000
LIMIT = 10000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=E3RGLK4WEGWRU1EJVZC1KHVAFKLG0D3HAY1TCMOMAV31IXAC&client_secret=LK2V3N010RFQMUJJJXRMQ4LADNDLBLKBYFU3LN3I2KWIGZRB&v=20200620&ll=51.518895,-0.078378&radius=1000&limit=10000'

In [84]:
results = requests.get(url).json()
#results

In [85]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [86]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Kastner & Ovens,Café,51.517913,-0.076465
1,Ottolenghi,Mediterranean Restaurant,51.518272,-0.077177
2,Old Spitalfields Market,Flea Market,51.519668,-0.075375
3,The Mayor of Scaredy Cat Town,Speakeasy,51.518524,-0.078882
4,Gunpowder,Indian Restaurant,51.518436,-0.074732


In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()['response'].get('groups',[{}])[0].get('items')
        results = requests.get(url).json()['response'].get('groups',[{}])[0].get('items', [])

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'latitude', 
                  'longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
nearby_venues

,name,categories,lat,lng
0,Kastner & Ovens,Café,51.517913,-0.076465
1,Ottolenghi,Mediterranean Restaurant,51.518272,-0.077177
2,Old Spitalfields Market,Flea Market,51.519668,-0.075375
3,The Mayor of Scaredy Cat Town,Speakeasy,51.518524,-0.078882
4,Gunpowder,Indian Restaurant,51.518436,-0.074732
5,Honest Burgers,Burger Joint,51.518011,-0.078038
6,Dumpling Shack,Food Truck,51.519586,-0.075403
7,Discount Suit Company,Cocktail Bar,51.516705,-0.075506
8,Merkamo Ethiopian,Food Stand,51.520099,-0.075230
9,Hawksmoor Spitalfields,Steakhouse,51.521438,-0.075844


In [89]:
london_venues = getNearbyVenues(names=df2['Postcode'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

E1 6AN
E1 7AA
E1 7AD
E1 7AE
E1 7AF
E1 7AW
E1 7AX
E1 7AY
E1 7BH
E1 7BS
E1 7DA
E1 7DB
E1 7DJ
E1 7EA
E1 7EB
E1 7ED
E1 7EE
E1 7EF
E1 7EG
E1 7ES
E1 7HP
E1 7HT
E1 7HY
E1 7JF
E1 7JH
E1 7JJ
E1 7LP
E1 7LS
E1 8AB
E1 8AE
E1 8AH
E1 8AL
E1 8AQ
E1 8AT
E1 8BQ
E1 8BT
E1 8BZ
E1 8PH
E1 8RB
EC1A 1HQ
EC1A 2AA
EC1A 2AE
EC1A 2AL
EC1A 2AT
EC1A 2AY
EC1A 2BN
EC1A 2BP
EC1A 2DE
EC1A 2DH
EC1A 2DJ
EC1A 2DP
EC1A 2DQ
EC1A 2EJ
EC1A 2FD
EC1A 2FG
EC1A 4AB
EC1A 4AS
EC1A 4BB
EC1A 4EE
EC1A 4EN
EC1A 4ER
EC1A 4EU
EC1A 4HD
EC1A 4HJ
EC1A 4HT
EC1A 4HU
EC1A 4HY
EC1A 4JA
EC1A 4JP
EC1A 4JR
EC1A 4LA
EC1A 4LX
EC1A 7AA
EC1A 7AB
EC1A 7AE
EC1A 7AF
EC1A 7AJ
EC1A 7AW
EC1A 7AZ
EC1A 7BA
EC1A 7BB
EC1A 7BD
EC1A 7BE
EC1A 7BF
EC1A 7BG
EC1A 7BH
EC1A 7BL
EC1A 7BN
EC1A 7BR
EC1A 7BT
EC1A 7BX
EC1A 7DH
EC1A 7EE
EC1A 7ES
EC1A 7HF
EC1A 7HH
EC1A 7HL
EC1A 7HN
EC1A 7HP
EC1A 7HT
EC1A 7HU
EC1A 7HW
EC1A 7HZ
EC1A 7JA
EC1A 7JN
EC1A 7JQ
EC1A 7LD
EC1A 7LP
EC1A 7LS
EC1A 7NT
EC1A 7NX
EC1A 7PH
EC1A 9BU
EC1A 9BW
EC1A 9DD
EC1A 9DE
EC1A 9DS
EC1A 9DY
EC1A 9EJ
EC1A 9E

In [13]:
london_venues.to_csv('london_venues.csv', header=False, index=False)

In [14]:
# Some idea of the new venues dataset
print(london_venues.shape)
london_venues.head()

(85250, 7)


,Name,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,E1 6AN,51.518895,-0.078378,Kastner & Ovens,51.517913,-0.076465,Café
1,E1 6AN,51.518895,-0.078378,Ottolenghi,51.518272,-0.077177,Mediterranean Restaurant
2,E1 6AN,51.518895,-0.078378,Old Spitalfields Market,51.519668,-0.075375,Flea Market
3,E1 6AN,51.518895,-0.078378,The Mayor of Scaredy Cat Town,51.518524,-0.078882,Speakeasy
4,E1 6AN,51.518895,-0.078378,Gunpowder,51.518436,-0.074732,Indian Restaurant


In [29]:
# Understand what categories have been returned from Foursquare API
london_venues["Venue Category"].value_counts()

Coffee Shop                      6789
Hotel                            6729
Gym / Fitness Center             5806
Pub                              3273
Garden                           2457
Scenic Lookout                   2265
French Restaurant                2103
Restaurant                       2036
Falafel Restaurant               1745
Cocktail Bar                     1700
Steakhouse                       1620
Event Space                      1569
Theater                          1554
Food Truck                       1528
Wine Bar                         1468
Roof Deck                        1444
Park                             1373
Boxing Gym                       1297
History Museum                   1294
Vietnamese Restaurant            1291
Salad Place                      1234
Grocery Store                     998
Street Food Gathering             938
South American Restaurant         934
Beer Bar                          911
Café                              899
Sushi Restau

In [30]:
# Remove non-restaurant venues
to_drop = ['Arcade', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store', 'Bagel Shop', 'Bakery', 'Bar', 'Beach', 'Beer Bar', 'Beer Store', 'Bike Shop', 'Bookstore', 'Botanical Garden', 'Boxing Gym', 'Breakfast Spot', 'Building', 'Butcher', 'Caf√©', 'Castle', 'Cheese Shop', 'Chocolate Shop', 'Church', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'Concert Hall', 'Cosmetics Shop', 'Coworking Space', 'Dance Studio', 'Deli / Bodega', 'Department Store', 'Distillery', 'Donut Shop', 'Electronics Store', 'Event Space', 'Exhibit', 'Farmers Market', 'Flea Market', 'Food Stand', 'Fruit & Vegetable Store', 'Furniture / Home Store', 'Garden', 'General Entertainment', 'Grocery Store', 'Gym', 'Gym / Fitness Center', 'Historic Site', 'History Museum', 'Hostel', 'Hotel', 'Hotel Bar', 'Ice Cream Shop', 'Indie Movie Theater', 'Juice Bar', 'Lounge', 'Market', 'Mini Golf', 'Monument / Landmark', 'Movie Theater', 'Museum', 'Music Venue', 'Office', 'Opera House', 'Organic Grocery', 'Park', 'Pedestrian Plaza', 'Performing Arts Venue', 'Pilates Studio', 'Plaza', 'Pub', 'Record Shop', 'Roof Deck', 'Scenic Lookout', 'Social Club', 'Souvenir Shop', 'Speakeasy', 'Sports Bar', 'Street Food Gathering', 'Supermarket', 'Tea Room', 'Theater', 'Trail', 'Whisky Bar', 'Wine Bar', 'Wine Shop']
london_restaurants = london_venues[~london_venues['Venue Category'].isin(to_drop)]

In [31]:
#Reset Index
london_restaurants = london_restaurants.reset_index()
del london_restaurants['index']

london_restaurants.rename(columns={'latitude':'Postcode Latitude',
                          'longitude':'Postcode Longitude'}, 
                 inplace=True)

#Show the top 10 rows
london_restaurants.head()

,Name,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,E1 6AN,51.518895,-0.078378,Kastner & Ovens,51.517913,-0.076465,Café
1,E1 6AN,51.518895,-0.078378,Ottolenghi,51.518272,-0.077177,Mediterranean Restaurant
2,E1 6AN,51.518895,-0.078378,Gunpowder,51.518436,-0.074732,Indian Restaurant
3,E1 6AN,51.518895,-0.078378,Honest Burgers,51.518011,-0.078038,Burger Joint
4,E1 6AN,51.518895,-0.078378,Dumpling Shack,51.519586,-0.075403,Food Truck


In [32]:
#Show the bottom 10 rows
london_restaurants.tail()

,Name,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23911,WC2A 1QU,51.517484,-0.112941,The Delaunay,51.513181,-0.117988,Restaurant
23912,WC2A 1QU,51.517484,-0.112941,Wop,51.521776,-0.110130,Food Truck
23913,WC2A 1QU,51.517484,-0.112941,Daddy Donkey,51.520551,-0.109418,Burrito Place
23914,WC2A 1QU,51.517484,-0.112941,eat TOKYO,51.519849,-0.117410,Japanese Restaurant
23915,WC2A 1QU,51.517484,-0.112941,Anglo,51.520536,-0.109351,Restaurant


In [33]:
london_restaurants.shape

(23916, 7)

In [54]:
london_restaurants["Summary Category"].value_counts()


Other Competitor                 4139
Falafel Restaurant               1745
Steakhouse                       1620
Food Truck                       1528
Pizza Competitor                 1312
Vietnamese Restaurant            1291
Salad Place                      1234
South American Restaurant         934
Café                              899
Sushi Restaurant                  871
Seafood Restaurant                753
Indian Restaurant                 714
Colombian Restaurant              662
Latin American Restaurant         642
Modern European Restaurant        614
Japanese Restaurant               526
Chinese Restaurant                453
Burger Joint                      442
Mediterranean Restaurant          442
Asian Restaurant                  415
Argentinian Restaurant            404
American Restaurant               332
Poke Place                        303
Portuguese Restaurant             277
Middle Eastern Restaurant         265
English Restaurant                262
Empanada Res

In [60]:
#Let's add a column to show Italian Restaurants and Pizza Places and the rest as Restaurants

# First add a column with the same values as Venue Category
london_restaurants['Summary Category'] = london_restaurants['Venue Category']

# Replace Italian Restaurants and Pizza Places with Pizza Competitor in Summary Category

london_restaurants["Summary Category"].replace({"Pizza Place": "Pizza Competitor", "Italian Restaurant": "Pizza Competitor"}, inplace=True)

# Replace All other Restaurants with Other Competitor in Summary Category
london_restaurants["Summary Category"].replace({"Restaurant": "Other Competitor", "French Restaurant": "Other Competitor", "Falafel Restaurant": "Other Competitor", "Steakhouse": "Other Competitor", "Food Truck": "Other Competitor", "Vietnamese Restaurant": "Other Competitor", "Salad Place": "Other Competitor", "South American Restaurant": "Other Competitor", "Café": "Other Competitor", "Sushi Restaurant": "Other Competitor", "Seafood Restaurant": "Other Competitor", "Indian Restaurant": "Other Competitor", "Colombian Restaurant": "Other Competitor", "Latin American Restaurant": "Other Competitor", "Modern European Restaurant": "Other Competitor", "Japanese Restaurant": "Other Competitor", "Chinese Restaurant": "Other Competitor", "Burger Joint": "Other Competitor", "Mediterranean Restaurant": "Other Competitor", "Asian Restaurant": "Other Competitor", "Argentinian Restaurant": "Other Competitor", "American Restaurant": "Other Competitor", "Poke Place": "Other Competitor", "Portuguese Restaurant": "Other Competitor", "Middle Eastern Restaurant": "Other Competitor", "English Restaurant": "Other Competitor", "Empanada Restaurant": "Other Competitor", "Taco Place": "Other Competitor", "Korean Restaurant": "Other Competitor", "Vegetarian / Vegan Restaurant": "Other Competitor", "Moroccan Restaurant": "Other Competitor", "BBQ Joint": "Other Competitor", "Burrito Place": "Other Competitor", "Fried Chicken Joint": "Other Competitor", "Turkish Restaurant": "Other Competitor", "Soup Place": "Other Competitor", "Fish & Chips Shop": "Other Competitor", "Sandwich Place": "Other Competitor", "Spanish Restaurant": "Other Competitor", "Herbs & Spices Store": "Other Competitor", "Lebanese Restaurant": "Other Competitor", }, inplace=True)
Check the Value Counts
london_restaurants["Summary Category"].value_counts()

Other Competitor    22604
Pizza Competitor     1312
Name: Summary Category, dtype: int64

In [66]:
# Describe the london_restaurants dataframe
london_restaurants.describe()
#Summarise by Post Code

,Postcode Latitude,Postcode Longitude,Venue Latitude,Venue Longitude
count,23916.000000,23916.000000,23916.000000,23916.000000
mean,51.514957,-0.090361,51.515383,-0.089937
std,0.003042,0.009980,0.004175,0.011113
min,51.508136,-0.113471,51.502856,-0.122371
25%,51.512552,-0.097295,51.513605,-0.098654
50%,51.514762,-0.088548,51.514750,-0.089687
75%,51.517082,-0.082253,51.518272,-0.080783
max,51.523000,-0.073262,51.526104,-0.068796


In [141]:
# Count the 
london_restaurants_grp = london_restaurants.groupby(['Name','Summary Category'],as_index=False).count()

In [142]:
london_restaurants_grp = pd.pivot_table(london_restaurants_grp, values = 'Postcode Latitude', index=['Name'], columns = 'Summary Category').reset_index()

In [143]:
london_restaurants_grp.fillna(0)

Summary Category,Name,Other Competitor,Pizza Competitor
0,E1 6AN,22.0,3.0
1,E1 7AA,18.0,1.0
2,E1 7AD,21.0,0.0
3,E1 7AE,20.0,0.0
4,E1 7AF,20.0,0.0
5,E1 7AW,20.0,1.0
6,E1 7AX,22.0,1.0
7,E1 7AY,22.0,1.0
8,E1 7BH,21.0,0.0
9,E1 7BS,21.0,0.0


In [113]:
# Add a column to the dataframe to give a score - based on a high score for # restaurants and subtracting the pizza competitiors
london_restaurants_grp['Net Score'] = london_restaurants_grp['Other Competitor'] - london_restaurants_grp['Pizza Competitor']

In [129]:
london_restaurants_grp.sort_values(by=['Net Score'], inplace=True, ascending=False)

In [133]:
Initial_selection = london_restaurants_grp.head(10)

In [137]:
print (Initial_selection)

Summary Category      Name  Other Competitor  Pizza Competitor  Net Score
23                  E1 7JF              24.0               1.0       23.0
22                  E1 7HY              24.0               1.0       23.0
277               EC2M 4QB              24.0               1.0       23.0
276               EC2M 4PT              24.0               1.0       23.0
354               EC2M 7PY              24.0               1.0       23.0
20                  E1 7HP              24.0               1.0       23.0
24                  E1 7JH              24.0               1.0       23.0
25                  E1 7JJ              24.0               1.0       23.0
27                  E1 7LS              24.0               1.0       23.0
248               EC2M 3TP              24.0               1.0       23.0
